Helpful to know exists - Request body:   
https://platform.openai.com/docs/api-reference/responses/create
- parallel tool calling: allow calling tools in parallel or not
- previous_response_id: allows for multi-turn
  - instructions: will not be carried over to next call if multi-turn activated
- include: can include files returned in search after tool call
- tool_choice: how the model should select which tool to choose when generating a response
- tools: the actual tools to call (web search, file search)

- gpt 4.1 nano is most cost effective for now

#### 

Goal: Review upcoming weather to determine when would be the best time period for someone to run.   
- Agent 1: Extract longitude and latitude coordinates from city in prompt, return weather 
- Agent 2: Evaluate when is best time period to run

### Import Credentials

In [ ]:
# !pip install openai
# !pip install requests

In [28]:
from creds import credentials
import calendars
import weather
import json
from openai import OpenAI
import os

KeyError: 'cronofy_access_token'

In [7]:
# get keys
openai_key = credentials['tool-calling-OpenAI']
weather_api = credentials['weather_api']

## Make tool
Must define function to use + how to use it

**Notes**  
When strict = True:
- all properties/params must be inputted in required list
- additionalProperties is False (what does this mean??)

## *Agent #1: Extract location and get weather from prompt*
- Goal: Get next 5 days of weather
- Practice: Setting up API call

In [10]:
weather_tool = [
    # weather tool
    {
        "type": "function",
        "name": "get_weather",
        "description":"get temperature, humidity, precipitation, wind speed for specified location for the next 5 days.",
        "parameters":{
            # param names + types
            "type": "object",
            "properties": {
                "lat": {"type":"number"},
                "long":{"type":"number"}
            },
            # all params in required b/c strict = True
            "required": ["lat", "long"],
            "additionalProperties": False
        },
    # recommended to always use true
    # does not allow model to skip parameteres
    "strict" : True
}]

In [11]:
client = OpenAI(api_key=openai_key)

input_message = [{
    "role":"user",
    "content": "What is the weather going to be in Cumberland, MD?"
}]

response = client.responses.create(
    model="gpt-4.1-nano",
    input=input_message,
    tools=weather_tool
)

print(response.output_text)

In [12]:
# response output includes these parts of tool: arguments, call_id, name, type, id, status

tool_call = response.output[0]
# get arguments feature of tool call
args = json.loads(tool_call.arguments)

# get_weather will print the lat and long
result = weather.get_weather(args["lat"], args["long"])

Lat: 39.6529, Long: -78.7625


In [14]:
results_dict = result.json()['data']
# check correct amount of intervals printed
len(results_dict['timelines'][0]['intervals'])

121

### *Agent #2: Get Upcoming calendar events*

In [ ]:
calendar_get_events = [
    # weather tool
    {
        "type": "function",
        "name": "get_upcoming_event",
        "description":"Get the days and times of when the user will be busy in the upcoming days. The furthest you can look ahead is 5 days.",
        "parameters":{
            # param names + types
            "type": "object",
            "properties": {
                "cal_id": {"type":"string",
                           "description":"ID of the calendar to use. Do not fill in if the user does not provide it, there is a default value in function."},
                "in_future":{"type":"number",
                             "description":"Number of days to look ahead. Do not fill in if the user does not provide it, there is a default value in function."}
            },
            # all params in required b/c strict = True
            "required": ["cal_id", "in_future"],
            "additionalProperties": False
        },
    # recommended to always use true
    # does not allow model to skip parameteres
    "strict" : True
}]

In [20]:
input_message = [{
    "role":"user",
    "content": "What days and times am I busy for the next 3 days?"
}]

response = client.responses.create(
    model="gpt-4.1-nano",
    input=input_message,
    tools=calendar_get_events
)

print(response.output_text)

### *Agent #3: Review the weather and evaluate when it would be best to run outside*

In [ ]:
input_message = [{
    "role":"user",
    "content": f"""The following information contains the weather for location with the coordinates of {args['lat']}, {args['long']}. 
    Here is the weather for each hour for the next five days: {results_dict['timelines'][0]['intervals']}. 
    What would be the best day and time to go for a 30 minute run?
    Do not give me an overview or analysis of the data. Just tell me the day(s) and time(s) the location time zone""",
}]

response = client.responses.create(
    model="gpt-4.1-nano",
    input=input_message
)

print(response.output_text)